<a href="https://colab.research.google.com/github/crescendonow/thai_geoparsing/blob/main/toponym_colab/IJG_2_CRF_attacut_Quadigram_PartOfGazetteer%2BExactMatch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm
!pip install pickle-mixin
!pip install sklearn_crfsuite
!pip install pythainlp[full]

In [ ]:
#library for use
from tqdm import tqdm
import pandas as pd
import re
import csv
import pickle
import codecs

#nlp library
from pythainlp import word_tokenize, Tokenizer
from pythainlp.tag import pos_tag, pos_tag_sents
from pythainlp.corpus.common import thai_stopwords
from nltk.tokenize import RegexpTokenizer

#Mange data with scikit-learn
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite import scorers,metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_validate, train_test_split

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/1.Tokennization/train_dn.data', 'rb') as token:
    train_data = pickle.load(token)
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/1.Tokennization/test_attacut_cl.data', 'rb') as token2:
    test_data = pickle.load(token2)

In [ ]:
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/acp_attacut.data', 'rb') as acp:
    academic = pickle.load(acp)
    academic = set(academic)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/acp_full.data', 'rb') as acpfull:
    academic_full = pickle.load(acpfull)
    academic_full = set(academic_full)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/admin_attacut.data', 'rb') as admin:
    admin = pickle.load(admin)
    admin = set(admin)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/bsn_attacut.data', 'rb') as bsn:
    bsn = pickle.load(bsn)
    bsn = set(bsn)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/bsn_full.data', 'rb') as bsnfull:
    bsn_full = pickle.load(bsnfull)
    bsn_full = set(bsn_full)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/fplace_attacut.data', 'rb') as fplace:
    fplace = pickle.load(fplace)
    fplace = set(fplace)

with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/gov_attacut.data', 'rb') as gov:
    gov = pickle.load(gov)
    gov = set(gov)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/gov_full.data', 'rb') as govfull:
    gov_full = pickle.load(govfull)
    gov_full = set(gov_full)

with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/hp_attacut.data', 'rb') as hp:
    hp = pickle.load(hp)
    hp = set(hp)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/hp_full.data', 'rb') as hpfull:
    hp_full = pickle.load(hpfull)
    hp_full = set(hp_full)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/mon_attacut.data', 'rb') as mon:
    mon = pickle.load(mon)
    mon = set(academic)

with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/nat_attacut.data', 'rb') as nat:
    nat = pickle.load(nat)
    nat = set(nat)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/rct_attacut.data', 'rb') as rct:
    rct = pickle.load(rct)
    rct = set(rct)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/rct_full.data', 'rb') as rctfull:
    rct_full = pickle.load(rctfull)
    rct_full = set(rct_full)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/res_attacut.data', 'rb') as res:
    res = pickle.load(res)
    res = set(res)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/res_full.data', 'rb') as resfull:
    res_full = pickle.load(resfull)
    res_full = set(res_full)

with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/rp_attacut.data', 'rb') as rp:
    rp = pickle.load(rp)
    rp = set(rp)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/rp_full.data', 'rb') as rpfull:
    rp_full = pickle.load(rpfull)
    rp_full = set(rp_full)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/store_attacut.data', 'rb') as store:
    store = pickle.load(store)
    store = set(store)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/store_full.data', 'rb') as storefull:
    store_full = pickle.load(storefull)
    store_full = set(store_full)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/tran_attacut.data', 'rb') as tran:
    tran = pickle.load(tran)
    tran = set(tran)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/tran_full.data', 'rb') as tranfull:
    tran_full = pickle.load(tranfull)
    tran_full = set(tran_full)
        
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/poiall_attacut.data', 'rb') as poiall:
    poiall = pickle.load(poiall)
    poiall = set(poiall)
    
with open('/content/drive/MyDrive/dissertation/TEXT DATA/TrainingData/2.Gazetteer/poi_full.data', 'rb') as poi_full:
    poi_full = pickle.load(poi_full)
    poi_full = set(poi_full)

In [ ]:
#Create feature function list
def prep_list(word):
    prep_list = set(['อยู่','อยู่ที่','ถึง','ไปถึง','มาถึง','มุ่งหน้า','เข้าสู่','เส้น','เยื้อง','ถัดไป','ถัดจาก','ตรงข้าม','ตรงกับ','ตรงข้ามกับ',
                'ติด','ติดกับ','ข้าง','ข้างหน้า'])
    if word in prep_list:
        return word
    else:
        return False

def prefix_nat(word):
    p_nat = set(['ภู','ภูเขา','ดอย','ทะเล','คลอง','แม่น้ำ','ห้วย','บึง'])
    if word in p_nat:
        return word
    else:
        return False

def prefix_res(word):
    p_res = set(['หมู่บ้าน','ม.','บ้าน','คอนโด','อาร์ตเมนต์'])
    if word in p_res:
        return word
    else:
        return False

def prefix_rp(word):
    p_rp  = set(['วัด','มัสยิด','โบสถ์','คริสตจักร'])
    if word in p_rp:
        return word
    else:
        return False

def prefix_acp(word):
    p_acp = set(['โรงเรียน','รร.','วิทยาลัย','มหาวิทยาลัย','ม.','สถาบัน','ราชภัฏ'])
    if word in p_acp:
        return word
    else:
        return False
    
def prefix_hp(word):
    p_hp = set(['โรงพยาบาล','คลินิค','ศูนย์ฑันตกรรม','โรงยิม','ส่งเสริม','สุขภาพ'])
    if word in p_hp:
        return word
    else:
        return False

def prefix_tran(word):
    p_tran = set(['สถานี','สถานีรถไฟ','บีทีเอส','bts','สนามบิน','ท่าอากาศยาน','ท่าเรือ'])
    if word in p_tran:
        return word
    else:
        return False

def prefix_store(word):
    p_store = set(['ร้าน','ตลาด'])
    if word in p_store:
        return word
    else:
        return False
    
def prefix_market(word):
    p_mkt = set(['ตลาด','ตลาดนัด'])
    if word in p_mkt:
        return word
    else:
        return False

def prefix_rt(word):
    p_rt = set(['ร้าน','ร้านอาหาร','ศูนย์อาหาร','ภัตตาคาร','ห้องอาหาร'])
    if  word in p_rt:
        return word
    else:
        return False

def prefix_dep(word):
    p_dep = set(['ห้างสรรพสินค้า','ศูนย์การค้า','เดอะมอลล์','เซ็นทรัล','บิ๊กซี','โลตัส'])
    if word in p_dep:
        return word
    else:
        return False

def prefix_bsn(word):
    p_bsn = set(['อาคาร','ตึก','สำนักงาน','บริษัท'])
    if word in p_bsn:
        return word
    else:
        return False

def prefix_gov(word):
    p_gov = set(['กระทรวง','ทบวง','กรม','ค่าย','ศาล','ศูนย์ราชการ','ศาลาว่าการ','สำนัก'])
    if word in p_gov:
        return word
    else:
        return False
    
def prefix_rct(word):
    p_rct = set(['สนามกีฬา','สวนสัตว์','สวนสนุก','สวนสาธารณะ','โรงภาพยนต์','โรงละคร','สตูดิโอ','พิพิธภัณฑ์'])
    if word in p_rct:
        return word
    else:
        return False
    
def prefix_mon(word):
    p_mon = set(['อนุเสาวรีย์','วงเวียน','หอนาฬิกา'])
    if word in p_mon:
        return word
    else:
        return False

def prefix_road(word):
    p_road = set(['ถนน','ถ.','ซอย','ซ.','ทางหลวง','ทางหลวงแผ่นดิน'])
    if word in p_road:
        return word
    else:
        return False

def prefix_admin(word):
    p_admin = set(['ตำบล','ต.','อำเภอ','อ.','จังหวัด','จ.','ประเทศไทย'])
    if word in p_admin:
        return word
    else:
        return False

def suffix_res(word):
    s_res = set(['ภิรมย์','ธานี','วิลเลจ','โฮม','การ์เดนโฮม','เลควิว','ธารา','ปาร์ควิว','วิลล์','เฮาส์'])
    if word in s_res:
        return word
    else:
        return False

def suffix_rp(word):
    s_rp = set(['วนาราม','วราราม','วรวิหาร','วรมหาวิหาร','อารามหลวง'])
    if word in s_rp:
        return word
    else:
        return False

def suffix_acp(word):
    s_acp = set(['วิทยา','วิทยาลัย','วิทยาคม','วิทยายน','วิทยาคาร','พิทยา','พิทยาลัย','พิทยาคม','ศึกษา','ศึกษาลัย'])
    if word in s_acp:
        return word
    else:
        return False

def suffix_store(word):
    s_store = set(['สโตร์','store','มาร์เก็ต','market'])
    if word in s_store:
        return word
    else:
        return False

def suffix_rt(word):
    s_res = set(['ภัตตาคาร','เรสเตอรองค์','restuarant','คาเฟ่','คาเฟ','cafe'])
    if word in s_res:
        return word
    else:
        return False

def suffix_dep(word):
    s_dep = set(['ดีพาร์ทเมนต์สโตร์','departmentstore','ซุเปอร์สโตร์','superstore','ซูปเปอร์เซ็นเตอร์','supercenter','เซ็นเตอร์','เซนเตอร์','center'])
    if word in s_dep:
        return word
    else:
        return False
    
#feature from word space 
def is_space(word):
    if word == ' ' or word == '\t' or word == '':
        return True
    else:
        return False
    
#len feature
def long_word(word):
    l_word = len(word)
    return l_word

#feature shape
def word_shape(word):
    t1 = re.sub('[A-Z]', 'E',word)
    t2 = re.sub('[a-z]', 'e',word)
    t3 = re.sub('[ก-๙]', 'T',word)
    return re.sub('[0-9]', 'd', t3)

In [ ]:
#Check in Gazetteer
def acp_gaz(word):
    if word in academic:
        return word
    else:
        return False

def acpfull_gaz(word):
    if word in academic_full:
        return word
    else:
        return False
    
def admin_gaz(word):
    if word in admin:
        return word
    else:
        return False
    
def bsn_gaz(word):
    if word in bsn:
        return word
    else:
        return False
    
def bsnfull_gaz(word):
    if word in bsn_full:
        return word
    else:
        return False
    
def fplace_gaz(word):
    if word in fplace:
        return word
    else:
        return False
    
def gov_gaz(word):
    if word in gov:
        return word
    else:
        return False
    
def govfull_gaz(word):
    if word in gov_full:
        return word
    else:
        return False
    
def hp_gaz(word):
    if word in hp:
        return word
    else:
        return False
    
def hpfull_gaz(word):
    if word in hp_full:
        return word
    else:
        return False
    
def mon_gaz(word):
    if word in mon:
        return word
    else:
        return False
    
def nat_gaz(word):
    if word in nat:
        return word
    else:
        return False

def rct_gaz(word):
    if word in rct:
        return word
    else:
        return False
    
def rctfull_gaz(word):
    if word in rct_full:
        return word
    else:
        return False
    
def res_gaz(word):
    if word in res:
        return word
    else:
        return False
    
def resfull_gaz(word):
    if word in res_full:
        return word
    else:
        return False 

def rp_gaz(word):
    if word in rp:
        return word
    else:
        return False
    
def rpfull_gaz(word):
    if word in rp_full:
        return word
    else:
        return False 

def store_gaz(word):
    if word in store:
        return word
    else:
        return False
    
def storefull_gaz(word):
    if word in store_full:
        return word
    else:
        return False 
    
def tran_gaz(word):
    if word in tran:
        return word
    else:
        return False
    
def tranfull_gaz(word):
    if word in tran_full:
        return word
    else:
        return False

def poiall_gaz(word):
    if word in poiall:
        return word
    else:
        return False
    
def poifull_gaz(word):
    if word in poi_full:
        return word
    else:
        return False

In [ ]:
def word_features(sentence,i):
    word = sentence[i][0]
    pos = sentence[i][1]
        
    #create feature in current word
    features = {
                    'bias': 1.0, 
                    'word': word,
                    'word[-3:]' : word[-3:],
                    'word[-2:]' : word[-2:],
                    'word.is_space()': is_space(word),
                    'word.long_word()' : long_word(word),
                    'word.shape_word()' : word_shape(word),
                    'word.prefix_nat()' : prefix_nat(word),
                    'word.prefix_res()' : prefix_res(word),
                    'word.prefix_rp()' : prefix_rp(word),
                    'word.prefix_acp()' : prefix_acp(word),
                    'word.prefix_hp()' : prefix_hp(word),
                    'word.prefix_tran()' : prefix_tran(word),
                    'word.prefix_store()' : prefix_store(word),
                    'word.prefix_mkt()' : prefix_market(word),
                    'word.prefix_rt()' : prefix_rt(word),
                    'word.prefix_dep()' : prefix_dep(word),
                    'word.prefix_bsn()' : prefix_bsn(word),
                    'word.prefix_gov()' : prefix_gov(word),
                    'word.prefix_rct()' : prefix_rct(word),
                    'word.prefix_mon()' : prefix_mon(word),
                    'word.prefix_road()' : prefix_road(word),
                    'word.prefix_admin()' : prefix_admin(word),
                    'word.prep_list()' : prep_list(word),
                    'word.acp_gaz()' : acp_gaz(word),
                    'word.acpfull_gaz()' : acpfull_gaz(word),
                    'word.admin_gaz()' : admin_gaz(word),
                    'word.bsn_gaz()' : bsn_gaz(word),
                    'word.bsnfull_gaz()' : bsnfull_gaz(word),
                    'word.fplace_gaz()' : fplace_gaz(word),
                    'word.gov_gaz()' : gov_gaz(word),
                    'word.govfull_gaz()' : govfull_gaz(word),
                    'word.hp_gaz()' : hp_gaz(word),
                    'word.hpfull_gaz()' : hpfull_gaz(word),
                    'word.mon_gaz()' : mon_gaz(word),
                    'word.nat_gaz()' : nat_gaz(word),
                    'word.rct_gaz()' : rct_gaz(word),
                    'word.rctfull_gaz()' : rctfull_gaz(word),
                    'word.res_gaz()' : res_gaz(word),
                    'word.resfull_gaz()' : resfull_gaz(word),
                    'word.rp_gaz()' : rp_gaz(word),
                    'word.rpfull_gaz()' : rpfull_gaz(word),
                    'word.store_gaz()' : store_gaz(word),
                    'word.storefull_gaz()' : storefull_gaz(word),
                    'word.tran_gaz()' : tran_gaz(word),
                    'word.tranfull_gaz()' : tranfull_gaz(word),
                    'word.poiall_gaz()' : poiall_gaz(word),
                    'word.poifull_gaz()' : poifull_gaz(word),
                    'postag': pos
                }
        #If this is not the first word of sentence 
    if i > 0:
        prev_word = sentence[i-1][0]
        prev_pos = sentence[i-1][1]
        features.update({
                            '-1:word' : prev_word,
                            '-1:word.is_space' : is_space(prev_word),
                            '-1:word.long_word()' : long_word(prev_word),
                            '-1:word.shape_word()' : word_shape(prev_word),
                            '-1:word.prefix_nat()' : prefix_nat(prev_word),
                            '-1:word.prefix_res()' : prefix_res(prev_word),
                            '-1:word.prefix_rp()' : prefix_rp(prev_word),
                            '-1:word.prefix_acp()' : prefix_acp(prev_word),
                            '-1:word.prefix_hp()' : prefix_hp(prev_word),
                            '-1:word.prefix_tran()' : prefix_tran(prev_word),
                            '-1:word.prefix_store()' : prefix_store(prev_word),
                            '-1:word.prefix_mkt()' : prefix_market(prev_word),
                            '-1:word.prefix_rt()' : prefix_rt(prev_word),
                            '-1:word.prefix_dep()' : prefix_dep(prev_word),
                            '-1:word.prefix_bsn()' : prefix_bsn(prev_word),
                            '-1:word.prefix_gov()' : prefix_gov(prev_word),
                            '-1:word.prefix_rct()' : prefix_rct(prev_word),
                            '-1:word.prefix_mon()' : prefix_mon(prev_word),
                            '-1:word.prefix_road()' : prefix_road(prev_word),
                            '-1:word.prefix_admin()' : prefix_admin(prev_word),
                            '-1:word.prep_list()' : prep_list(prev_word),
                            '-1:word.acp_gaz()' : acp_gaz(prev_word),
                            '-1:word.acpfull_gaz()' : acpfull_gaz(prev_word),
                            '-1:word.admin_gaz()' : admin_gaz(prev_word),
                            '-1:word.bsn_gaz()' : bsn_gaz(prev_word),
                            '-1:word.bsnfull_gaz()' : bsnfull_gaz(prev_word),
                            '-1:word.fplace_gaz()' : fplace_gaz(prev_word),
                            '-1:word.gov_gaz()' : gov_gaz(prev_word),
                            '-1:word.govfull_gaz()' : govfull_gaz(prev_word),
                            '-1:word.hp_gaz()' : hp_gaz(prev_word),
                            '-1:word.hpfull_gaz()' : hpfull_gaz(prev_word),
                            '-1:word.mon_gaz()' : mon_gaz(prev_word),
                            '-1:word.nat_gaz()' : nat_gaz(prev_word),
                            '-1:word.rct_gaz()' : rct_gaz(prev_word),
                            '-1:word.rctfull_gaz()' : rctfull_gaz(prev_word),
                            '-1:word.res_gaz()' : res_gaz(prev_word),
                            '-1:word.resfull_gaz()' : resfull_gaz(prev_word),
                            '-1:word.rp_gaz()' : rp_gaz(prev_word),
                            '-1:word.rpfull_gaz()' : rpfull_gaz(prev_word),
                            '-1:word.store_gaz()' : store_gaz(prev_word),
                            '-1:word.storefull_gaz()' : storefull_gaz(prev_word),
                            '-1:word.tran_gaz()' : tran_gaz(prev_word),
                            '-1:word.tranfull_gaz()' : tranfull_gaz(prev_word),
                            '-1:word.poiall_gaz()' : poiall_gaz(prev_word),
                            '-1:word.poifull_gaz()' : poifull_gaz(prev_word),
                            '-1:postag' : prev_pos
                        })
    else:
        features['BOS'] = True
        
    if i < len(sentence)-1:
        next_word = sentence[i+1][0]
        next_pos = sentence[i+1][0]
        features.update({
                            '+1:word' : next_word,
                            '+1:word.is_space' : is_space(next_word),
                            '+1:word.long_word()' : long_word(next_word),
                            '+1:word.suffix_res()' : suffix_res(next_word),
                            '+1:word.suffix_rp()' : suffix_rp(next_word),
                            '+1:word.suffix_acp()' : suffix_acp(next_word),
                            '+1:word.suffix_store()' : suffix_store(next_word),
                            '+1:word.suffix_rt()' : suffix_rt(next_word),
                            '+1:word.suffix_dep()' : suffix_dep(next_word),
                            '+1:word.acp_gaz()' : acp_gaz(next_word),
                            '+1:word.acpfull_gaz()' : acpfull_gaz(next_word),
                            '+1:word.admin_gaz()' : admin_gaz(next_word),
                            '+1:word.bsn_gaz()' : bsn_gaz(next_word),
                            '+1:word.bsnfull_gaz()' : bsnfull_gaz(next_word),
                            '+1:word.fplace_gaz()' : fplace_gaz(next_word),
                            '+1:word.gov_gaz()' : gov_gaz(next_word),
                            '+1:word.govfull_gaz()' : govfull_gaz(next_word),
                            '+1:word.hp_gaz()' : hp_gaz(next_word),
                            '+1:word.hpfull_gaz()' : hpfull_gaz(next_word),
                            '+1:word.mon_gaz()' : mon_gaz(next_word),
                            '+1:word.nat_gaz()' : nat_gaz(next_word),
                            '+1:word.rct_gaz()' : rct_gaz(next_word),
                            '+1:word.rctfull_gaz()' : rctfull_gaz(next_word),
                            '+1:word.res_gaz()' : res_gaz(next_word),
                            '+1:word.resfull_gaz()' : resfull_gaz(next_word),
                            '+1:word.rp_gaz()' : rp_gaz(next_word),
                            '+1:word.rpfull_gaz()' : rpfull_gaz(next_word),
                            '+1:word.store_gaz()' : store_gaz(next_word),
                            '+1:word.storefull_gaz()' : storefull_gaz(next_word),
                            '+1:word.tran_gaz()' : tran_gaz(next_word),
                            '+1:word.tranfull_gaz()' : tranfull_gaz(next_word),
                            '+1word.poiall_gaz()' : poiall_gaz(next_word),
                            '+1:word.poifull_gaz()' : poifull_gaz(next_word),
                            '+1:postag' : next_pos
                        })
    else:
        features['EOS'] = True
        
    if i > 1:
        prev_word2 = sentence[i-2][0]
        prev_pos2 = sentence[i-2][1]
        features.update({
                           
                            '-2:word' : prev_word2,
                            '-2:word.is_space' : is_space(prev_word2),
                            '-2:word.long_word()' : long_word(prev_word2),
                            '-2:word.shape_word()' : word_shape(prev_word2),
                            '-2:word.prefix_nat()' : prefix_nat(prev_word2),
                            '-2:word.prefix_res()' : prefix_res(prev_word2),
                            '-2:word.prefix_rp()' : prefix_rp(prev_word2),
                            '-2:word.prefix_acp()' : prefix_acp(prev_word2),
                            '-2:word.prefix_hp()' : prefix_hp(prev_word2),
                            '-2:word.prefix_tran()' : prefix_tran(prev_word2),
                            '-2:word.prefix_store()' : prefix_store(prev_word2),
                            '-2:word.prefix_mkt()' : prefix_market(prev_word2),
                            '-2:word.prefix_rt()' : prefix_rt(prev_word2),
                            '-2:word.prefix_dep()' : prefix_dep(prev_word2),
                            '-2:word.prefix_bsn()' : prefix_bsn(prev_word2),
                            '-2:word.prefix_gov()' : prefix_gov(prev_word2),
                            '-2:word.prefix_rct()' : prefix_rct(prev_word2),
                            '-2:word.prefix_mon()' : prefix_mon(prev_word2),
                            '-2:word.prefix_road()' : prefix_road(prev_word2),
                            '-2:word.prefix_admin()' : prefix_admin(prev_word2),
                            '-2:word.prep_list()' : prep_list(prev_word2),
                            '-2:word.acp_gaz()' : acp_gaz(prev_word2),
                            '-2:word.acpfull_gaz()' : acpfull_gaz(prev_word2),
                            '-2:word.admin_gaz()' : admin_gaz(prev_word2),
                            '-2:word.bsn_gaz()' : bsn_gaz(prev_word2),
                            '-2:word.bsnfull_gaz()' : bsnfull_gaz(prev_word2),
                            '-2:word.fplace_gaz()' : fplace_gaz(prev_word2),
                            '-2:word.gov_gaz()' : gov_gaz(prev_word2),
                            '-2:word.govfull_gaz()' : govfull_gaz(prev_word2),
                            '-2:word.hp_gaz()' : hp_gaz(prev_word2),
                            '-2:word.hpfull_gaz()' : hpfull_gaz(prev_word2),
                            '-2:word.mon_gaz()' : mon_gaz(prev_word2),
                            '-2:word.nat_gaz()' : nat_gaz(prev_word2),
                            '-2:word.rct_gaz()' : rct_gaz(prev_word2),
                            '-2:word.rctfull_gaz()' : rctfull_gaz(prev_word2),
                            '-2:word.res_gaz()' : res_gaz(prev_word2),
                            '-2:word.resfull_gaz()' : resfull_gaz(prev_word2),
                            '-2:word.rp_gaz()' : rp_gaz(prev_word2),
                            '-2:word.rpfull_gaz()' : rpfull_gaz(prev_word2),
                            '-2:word.store_gaz()' : store_gaz(prev_word2),
                            '-2:word.storefull_gaz()' : storefull_gaz(prev_word2),
                            '-2:word.tran_gaz()' : tran_gaz(prev_word2),
                            '-2:word.tranfull_gaz()' : tranfull_gaz(prev_word2),
                            '-2:word.poiall_gaz()' : poiall_gaz(prev_word2),
                            '-2:word.poifull_gaz()' : poifull_gaz(prev_word2),
                            '-2:postag' : prev_pos2
                        })
    else:
        features['BOS'] = True
   
    if i < len(sentence)-2:
        next_word2 = sentence[i+2][0]
        next_pos2 = sentence[i+2][0]
        features.update({
                           
                            '+2:word' : next_word2,
                            '+2:word.is_space' : is_space(next_word2),
                            '+2:word.long_word()' : long_word(next_word2),
                            '+2:word.shape_word()' : word_shape(next_word2),
                            '+2:word.suffix_res()' : suffix_res(next_word2),
                            '+2:word.suffix_rp()' : suffix_rp(next_word2),
                            '+2:word.suffix_acp()' : suffix_acp(next_word2),
                            '+2:word.suffix_store()' : suffix_store(next_word2),
                            '+2:word.suffix_rt()' : suffix_rt(next_word2),
                            '+2:word.suffix_dep()' : suffix_dep(next_word2),
                            '+2:word.acp_gaz()' : acp_gaz(next_word2),
                            '+2:word.acpfull_gaz()' : acpfull_gaz(next_word2),
                            '+2:word.admin_gaz()' : admin_gaz(next_word2),
                            '+2:word.bsn_gaz()' : bsn_gaz(next_word2),
                            '+2:word.bsnfull_gaz()' : bsnfull_gaz(next_word2),
                            '+2:word.fplace_gaz()' : fplace_gaz(next_word2),
                            '+2:word.gov_gaz()' : gov_gaz(next_word2),
                            '+2:word.govfull_gaz()' : govfull_gaz(next_word2),
                            '+2:word.hp_gaz()' : hp_gaz(next_word2),
                            '+2:word.hpfull_gaz()' : hpfull_gaz(next_word2),
                            '+2:word.mon_gaz()' : mon_gaz(next_word2),
                            '+2:word.nat_gaz()' : nat_gaz(next_word2),
                            '+2:word.rct_gaz()' : rct_gaz(next_word2),
                            '+2:word.rctfull_gaz()' : rctfull_gaz(next_word2),
                            '+2:word.res_gaz()' : res_gaz(next_word2),
                            '+2:word.resfull_gaz()' : resfull_gaz(next_word2),
                            '+2:word.rp_gaz()' : rp_gaz(next_word2),
                            '+2:word.rpfull_gaz()' : rpfull_gaz(next_word2),
                            '+2:word.store_gaz()' : store_gaz(next_word2),
                            '+2:word.storefull_gaz()' : storefull_gaz(next_word2),
                            '+2:word.tran_gaz()' : tran_gaz(next_word2),
                            '+2:word.tranfull_gaz()' : tranfull_gaz(next_word2),
                            '+2:word.poiall_gaz()' : poiall_gaz(next_word2),
                            '+2:word.poifull_gaz()' : poifull_gaz(next_word2),
                            '+2:postag' : next_pos2
                        })
    else:
        features['EOS'] = True
        
    if i > 2:
        prev_word3 = sentence[i-3][0]
        prev_pos3 = sentence[i-3][1]
        features.update({
                            '-3:word' : prev_word3,
                            '-3:word.is_space' : is_space(prev_word3),
                            '-3:word.long_word()' : long_word(prev_word3),
                            '-3:word.shape_word()' : word_shape(prev_word3),
                            '-3:word.prefix_nat()' : prefix_nat(prev_word3),
                            '-3:word.prefix_res()' : prefix_res(prev_word3),
                            '-3:word.prefix_rp()' : prefix_rp(prev_word3),
                            '-3:word.prefix_acp()' : prefix_acp(prev_word3),
                            '-3:word.prefix_hp()' : prefix_hp(prev_word3),
                            '-3:word.prefix_tran()' : prefix_tran(prev_word3),
                            '-3:word.prefix_store()' : prefix_store(prev_word3),
                            '-3:word.prefix_mkt()' : prefix_market(prev_word3),
                            '-3:word.prefix_rt()' : prefix_rt(prev_word3),
                            '-3:word.prefix_dep()' : prefix_dep(prev_word3),
                            '-3:word.prefix_bsn()' : prefix_bsn(prev_word3),
                            '-3:word.prefix_gov()' : prefix_gov(prev_word3),
                            '-3:word.prefix_rct()' : prefix_rct(prev_word3),
                            '-3:word.prefix_mon()' : prefix_mon(prev_word3),
                            '-3:word.prefix_road()' : prefix_road(prev_word3),
                            '-3:word.prefix_admin()' : prefix_admin(prev_word3),
                            '-3:word.prep_list()' : prep_list(prev_word3),
                            '-3:word.acp_gaz()' : acp_gaz(prev_word3),
                            '-3:word.acpfull_gaz()' : acpfull_gaz(prev_word3),
                            '-3:word.admin_gaz()' : admin_gaz(prev_word3),
                            '-3:word.bsn_gaz()' : bsn_gaz(prev_word3),
                            '-3:word.bsnfull_gaz()' : bsnfull_gaz(prev_word3),
                            '-3:word.fplace_gaz()' : fplace_gaz(prev_word3),
                            '-3:word.gov_gaz()' : gov_gaz(prev_word3),
                            '-3:word.govfull_gaz()' : govfull_gaz(prev_word3),
                            '-3:word.hp_gaz()' : hp_gaz(prev_word3),
                            '-3:word.hpfull_gaz()' : hpfull_gaz(prev_word3),
                            '-3:word.mon_gaz()' : mon_gaz(prev_word3),
                            '-3:word.nat_gaz()' : nat_gaz(prev_word3),
                            '-3:word.rct_gaz()' : rct_gaz(prev_word3),
                            '-3:word.rctfull_gaz()' : rctfull_gaz(prev_word3),
                            '-3:word.res_gaz()' : res_gaz(prev_word3),
                            '-3:word.resfull_gaz()' : resfull_gaz(prev_word3),
                            '-3:word.rp_gaz()' : rp_gaz(prev_word3),
                            '-3:word.rpfull_gaz()' : rpfull_gaz(prev_word3),
                            '-3:word.store_gaz()' : store_gaz(prev_word3),
                            '-3:word.storefull_gaz()' : storefull_gaz(prev_word3),
                            '-3:word.tran_gaz()' : tran_gaz(prev_word3),
                            '-3:word.tranfull_gaz()' : tranfull_gaz(prev_word3),
                            '-3:word.poiall_gaz()' : poiall_gaz(prev_word3),
                            '-3:word.poifull_gaz()' : poifull_gaz(prev_word3),
                            '-3:postag' : prev_pos3
                        })
    else:
        features['EOS'] = True
        
    
    if i < len(sentence)-3:
        next_word3 = sentence[i+3][0]
        next_pos3 = sentence[i+3][0]
        features.update({
                            '+3:word' : next_word3,
                            '+3:word.is_space' : is_space(next_word3),
                            '+3:word.long_word()' : long_word(next_word3),
                            '+3:word.shape_word()' : word_shape(next_word3),
                            '+3:word.suffix_res()' : suffix_res(next_word3),
                            '+3:word.suffix_rp()' : suffix_rp(next_word3),
                            '+3:word.suffix_acp()' : suffix_acp(next_word3),
                            '+3:word.suffix_store()' : suffix_store(next_word3),
                            '+3:word.suffix_rt()' : suffix_rt(next_word3),
                            '+3:word.suffix_dep()' : suffix_dep(next_word3),
                            '+3:word.acp_gaz()' : acp_gaz(next_word3),
                            '+3:word.acpfull_gaz()' : acpfull_gaz(next_word3),
                            '+3:word.admin_gaz()' : admin_gaz(next_word3),
                            '+3:word.bsn_gaz()' : bsn_gaz(next_word3),
                            '+3:word.bsnfull_gaz()' : bsnfull_gaz(next_word3),
                            '+3:word.fplace_gaz()' : fplace_gaz(next_word3),
                            '+3:word.gov_gaz()' : gov_gaz(next_word3),
                            '+3:word.govfull_gaz()' : govfull_gaz(next_word3),
                            '+3:word.hp_gaz()' : hp_gaz(next_word3),
                            '+3:word.hpfull_gaz()' : hpfull_gaz(next_word3),
                            '+3:word.mon_gaz()' : mon_gaz(next_word3),
                            '+3:word.nat_gaz()' : nat_gaz(next_word3),
                            '+3:word.rct_gaz()' : rct_gaz(next_word3),
                            '+3:word.rctfull_gaz()' : rctfull_gaz(next_word3),
                            '+3:word.res_gaz()' : res_gaz(next_word3),
                            '+3:word.resfull_gaz()' : resfull_gaz(next_word3),
                            '+3:word.rp_gaz()' : rp_gaz(next_word3),
                            '+3:word.rpfull_gaz()' : rpfull_gaz(next_word3),
                            '+3:word.store_gaz()' : store_gaz(next_word3),
                            '+3:word.storefull_gaz()' : storefull_gaz(next_word3),
                            '+3:word.tran_gaz()' : tran_gaz(next_word3),
                            '+3:word.tranfull_gaz()' : tranfull_gaz(next_word3),
                            '+3:word.poiall_gaz()' : poiall_gaz(next_word3),
                            '+3:word.poifull_gaz()' : poifull_gaz(next_word3),
                            '+3:postag' : next_pos3
                        })
    else:
        features['EOS'] = True  
        
    return features
                
#return feature dictionary for each word
def sentence_features(sentence):
    return [word_features(sentence,i) for i in range(len(sentence))]

#return the label NER tags
def sentence_labels(sentence):
    return [label for token,pos,label in sentence]

#return token 
def sentence_tokens(sentence):
    return [token for token,pos,label in sentence]

In [ ]:
%%time
#Prepare word for train test
X = [sentence_features(sentence) for sentence in tqdm(train_data)]
X_test = [sentence_features(sentence) for sentence in tqdm(test_data)]

#Label train test
y = [sentence_labels(sentence) for sentence in tqdm(train_data)]
y_test = [sentence_labels(sentence) for sentence in tqdm(test_data)]

#Get token 
Train_token = [sentence_tokens(sentence) for sentence in tqdm(train_data)]
Test_tokens = [sentence_tokens(sentence) for sentence in tqdm(test_data)]

100%|██████████| 5617/5617 [00:00<00:00, 136505.84it/s]

CPU times: user 1min 6s, sys: 11.8 s, total: 1min 17s
Wall time: 1min 18s


In [ ]:
%%time
#Train a CRF Model
crf = CRF(algorithm='lbfgs',
          c1=0.1,
          c2=0.1,
          max_iterations=200,
          all_possible_transitions=True,
          verbose=True)
# Train the CRF model on the supplied training data
crf.fit(X, y)

loading training data to CRFsuite: 100%|██████████| 17955/17955 [02:01<00:00, 148.06it/s]



Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 647667
Seconds required: 17.119

L-BFGS optimization
c1: 0.100000
c2: 0.100000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

Iter 1   time=35.68 loss=138269.68 active=634788 feature_norm=1.00
Iter 2   time=17.75 loss=137603.25 active=615470 feature_norm=0.99
Iter 3   time=17.84 loss=125364.29 active=260154 feature_norm=1.08
Iter 4   time=18.57 loss=116009.06 active=352604 feature_norm=1.14
Iter 5   time=19.06 loss=108854.98 active=365861 feature_norm=1.35
Iter 6   time=18.02 loss=89632.08 active=325423 feature_norm=2.40
Iter 7   time=35.70 loss=81337.15 active=333722 feature_norm=2.66
Iter 8   time=17.90 loss=78257.80 active=355252 feature_norm=3.41
Iter 9   time=35.90 loss=73414.92 active=354466 feature_norm=3.4

In [ ]:
labels = list(crf.classes_)
labels.remove('O')

In [ ]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred,
                      average='weighted', labels=labels)

0.8657741211968164

In [ ]:
# group B and I results
sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

In [ ]:
def  chunk_extract(tag_list,sentence_id):
  """
  >>> chunk_extract(['O', 'B-PER', 'I-PER', 'B-ORG', 'O'], 1)
  [(1, 1, 3, 'PER'), (1, 3, 4, 'ORG')]
  """
  nes = []
  for i, tag in enumerate(tag_list):
    if tag[0] == 'B':
        ner_type = tag[2:]
        if i < len(tag_list)-1:  
            current = i + 1
            if tag_list[current] == 'O':
                nes.append((sentence_id,i,i, ner_type)) 
            elif tag_list[current] == 'I-{}'.format(ner_type):    
                while tag_list[current] == 'I-{}'.format(ner_type) and current < (len(tag_list)-1):
                    current += 1
                nes.append((sentence_id,i, current, ner_type))
        else:
            nes.append((sentence_id,i,i, ner_type))
  return nes

In [ ]:
sentence_id = [x for x in range(len(y_test))]

In [ ]:
chunk_test = set()
error_id = []
for sent,idx in tqdm(zip(y_test,sentence_id)):
    ch = chunk_extract(sent,idx)
    chunk_test.update(ch)

5617it [00:00, 175214.98it/s]


In [ ]:
chunk_pred = set()
error_id = []
for sent,idx in tqdm(zip(y_pred,sentence_id)):
    ch = chunk_extract(sent,idx)
    chunk_pred.update(ch)

5617it [00:00, 145095.25it/s]


In [ ]:
def evaluation_phrase(true,prediction):
    total_correct = len(true.intersection(prediction))
    total_predict = len(prediction)
    total_true = len(true)
    
    precision = total_correct/total_predict
    recall = total_correct/total_true
    f1 = (2 * precision * recall)/(precision + recall)
    
    print('total_correct:',total_correct,':','total_predict:',total_predict,':','total_true:',total_true)
    print('precision:', round(precision,3))
    print('recall:', round(recall,3))
    print('f1:', round(f1,3))
    return [round(precision,3),round(recall,3),round(f1,3)]

In [ ]:
evaluation_phrase(chunk_test,chunk_pred)

total_correct: 928 : total_predict: 1026 : total_true: 1129
precision: 0.904
recall: 0.822
f1: 0.861


[0.904, 0.822, 0.861]

In [ ]:
def evaluation_phrase_type(chunk_test,chunk_pred):
    total_pred, total_true, total_correct = 0, 0, 0
    df = pd.DataFrame(columns=['PRECISION','RECALL','F1','SUPPORT'])
    ent_types = ['ACP','ADMIN','BSN','DEP','FPLACE','GOV','HP','MKT','MON','NAT','OTHER','RCT','RES','ROAD','RP','RT','STORE','TRAN']
    list_correct = []
    for ent in ent_types:
        true_set = []
        pred_set = []
        type_pred,type_true,type_correct = 0,0,0
        for tag in chunk_test:
            if tag[3] == ent:
                true_set.append(tag)
            if ent in tag:
                type_true+=1
                tag_true = set(true_set)
        for tag2 in chunk_pred:
            if tag2[3] == ent:
                pred_set.append(tag2)
            if ent in tag2:
                type_pred+=1
                tag_pred = set(pred_set)
        type_correct=len(tag_true.intersection(tag_pred))
        try:
            precision = type_correct/type_pred
        except:
            precision = 0
        recall = type_correct/type_true
        try:
            f1 = (2 * precision * recall) / (precision + recall)
        except:
            f1 = 0
        list_correct.append([type_true,type_pred,type_correct,ent])
        df.loc[ent] = [round(precision,3), round(recall,3), round(f1,3), str(type_true)]

    #Calculate micro macro f1
    total_true,total_pred,total_correct = 0,0,0
    for p in list_correct:
        total_true+=p[0]
        total_pred+=p[1]
        total_correct+=p[2]
    precision_micro = total_correct / total_pred
    recall_micro = total_correct / total_true
    f1_micro = (2 * precision_micro * recall_micro) / (precision_micro + recall_micro)
    df.loc['MACRO'] = [round(df.PRECISION.mean(),3), round(df.RECALL.mean(),3), round(df.F1.mean(),3), str(total_true)]
    df.loc['MICRO'] = [round(precision_micro,3), round(recall_micro,3), round(f1_micro,3), str(total_true)]

    print(df)

In [ ]:
chunk_test = list(chunk_test)
chunk_pred = list(chunk_pred)

In [ ]:
evaluation_phrase_type(chunk_test,chunk_pred)

        PRECISION  RECALL     F1 SUPPORT
ACP         0.906   0.784  0.841      37
ADMIN       0.934   0.916  0.925     309
BSN         0.913   0.808  0.857      26
DEP         0.872   0.728  0.794     103
FPLACE      0.948   0.737  0.830      99
GOV         1.000   0.680  0.810      25
HP          0.900   0.750  0.818      24
MKT         0.947   1.000  0.973      18
MON         1.000   0.500  0.667       2
NAT         1.000   0.909  0.952      11
OTHER       1.000   1.000  1.000       3
RCT         0.821   0.727  0.771      44
RES         0.920   0.767  0.836      30
ROAD        0.896   0.832  0.863     155
RP          0.909   0.784  0.842      51
RT          0.838   0.845  0.842     129
STORE       0.895   0.708  0.791      24
TRAN        0.882   0.769  0.822      39
MACRO       0.921   0.791  0.846    1129
MICRO       0.904   0.822  0.861    1129


In [ ]:
#save model
with open(r'D:\TEXT DATA\TrainingData\3.Model\1.14CRF_attacut_Quadigram_PartOfGazetter+ExactMatch.model', 'wb') as model:
    pickle.dump(crf,model)